# M.Lytova, M.Spanner, I.Tamblyn. *Deep learning and high harmonic generation* (2020)
## Codes for Section IV.A.1 : *Symmetric diatomic molecule* (spectra)

##Headers and constants

In [ ]:
from google.colab import files
import numpy as np
import tensorflow as tf
from keras.layers import Input, Dense, Dropout, Conv1D, MaxPooling1D, UpSampling1D, Flatten, Reshape
from keras.models import Model
from keras.optimizers import Nadam, Adam
from tensorflow.keras import initializers
from keras import objectives
from keras.losses import mean_squared_error
from keras.callbacks import TensorBoard
from keras import backend as K
import argparse
import matplotlib.pyplot as plt
import time

In [ ]:
PI = 3.14159265359

w_n_points = 1024   # number of nodes in frequency
path2load = f"/hhg_fft2/omega.dat"
load_w = np.loadtxt(path2load)
w_n = load_w[0:w_n_points]      # grid in frequency

n_train = 30000     # training set size
n_test = 1000       # testing set size

##Loading a training set

In [ ]:
param_train = np.zeros((n_train, 3))
path2param = f"/hhg_fft2/param.dat"
param_train = np.loadtxt(path2param, delimiter = ",", max_rows = n_train) 

In [ ]:
y_train = np.zeros((n_train, w_n_points))
path2load0 = f"/hhg_fft2/hhg"

tic = time.perf_counter()

for i in range(n_train): 
    path2load = path2load0 + str(i+1) + '.dat'    
    load_data = np.loadtxt(path2load)
    y_train[i] = load_data[0:w_n_points]
    if (round(i/1000)==i/1000):
        print(i)        

toc = time.perf_counter()
print(f"Training set preparation time {toc - tic:0.4f} seconds")   

In [ ]:
def plot_train_example(i1, i2):
    plt.figure(figsize=(20,10), constrained_layout=False)   
    plt.subplot(211)  
    plt.title('theta = ' + str(round(param_train[i1,0]*180/PI,2)) + ",  R = " + str(round(param_train[i1,1],2)) + \
              "a.u. , I = " + str(round((param_train[i1,2]/5.338027e-2)**2,2)) + "e14 W/cm^2", fontsize=16)   
    plt.plot(w_n, np.log10(y_train[i1]), color='green')
    plt.xlabel('$\omega/\omega_0$', fontsize=14)
    plt.ylabel('$S(\omega)$', fontsize=14)
    plt.xticks(np.arange(1, Wmax, 2.0))
    plt.grid()
    plt.subplot(212)
    plt.title('theta = ' + str(round(param_train[i2,0]*180/PI,2)) + ",  R = " + str(round(param_train[i2,1],2)) + \
              "a.u. , I = " + str(round((param_train[i2,2]/5.338027e-2)**2,2)) + "e14 W/cm^2", fontsize=16)
    plt.plot(w_n, np.log10(y_train[i2]), color='green')
    plt.xlabel('$\omega/\omega_0$', fontsize=14)
    plt.ylabel('$S(\omega)$', fontsize=14)
    plt.xticks(np.arange(1, Wmax, 2.0))   
    plt.grid()    
    plt.show() 
    plt.close()

### Drawing of a randomly chosen $S_k(\omega)$

In [ ]:
i_show1 = np.random.randint(0, n_train-1)
i_show2 = np.random.randint(0, n_train-1)

plot_train_example(i_show1, i_show2)

##Loading a testing set

In [ ]:
param_test = np.zeros((n_test, 3))
path2param = f"/hhg_fft2/param.dat"
param_test = np.loadtxt(path2param, delimiter = ",", skiprows = n_train, max_rows = n_test) 

In [ ]:
y_test = np.zeros((n_test, w_n_points))
path2load0 = f"/hhg_fft2/hhg"

for i in range(n_test): 
    path2load = path2load0 + str(i+1+n_train) + '.dat'    
    load_data = np.loadtxt(path2load)
    y_test[i] = load_data[0:w_n_points]      

##Normalizing before training

In [ ]:
max_E0 = np.amax(param_train[:,2])
min_E0 = np.amin(param_train[:,2])
param_train_norm = (param_train-[0, 1.5, min_E0])/[PI/2, 2.5, (max_E0-min_E0)]
param_test_norm = (param_test-[0, 1.5, min_E0])/[PI/2, 2.5, (max_E0-min_E0)]

w_n_half = int(3*w_n_points/4)
s_min =  1e-16

y_train_aux = np.log10(y_train[:,0:w_n_half]/s_min)
s_max = 16 

y_train_norm = np.zeros((n_train, w_n_half))
y_test_norm = np.zeros((n_test, w_n_half))

y_train_norm = y_train_aux/s_max
y_test_norm = np.log10(y_test[:,0:w_n_half]/s_min)/s_max

In [ ]:
def plot_train_norm_example(i1,i2):
    plt.figure(figsize=(20,10), constrained_layout=False)       
    plt.subplot(211)  
    plt.title('theta = ' + str(round(param_train[i1,0]*180/PI,2)) + ",  R = " + str(round(param_train[i1,1],2)) + \
              "a.u. , I = " + str(round((param_train[i1,2]/5.338027e-2)**2,2)) + "e14 W/cm^2", fontsize=16)
    plt.plot(w_n[0:w_n_half], y_train_norm[i1], color='green')
    plt.xlabel('$\omega/\omega_0$', fontsize=14)
    plt.xlim((0, 70))
    plt.ylim((0, 1))
    plt.ylabel('$S(\omega)$', fontsize=14)
    plt.xticks(np.arange(1, 3*Wmax/4, 2.0))
    plt.grid()
    plt.subplot(212)
    plt.title('theta = ' + str(round(param_train[i2,0]*180/PI,2)) + ",  R = " + str(round(param_train[i2,1],2)) + \
              "a.u. , I = " + str(round((param_train[i2,2]/5.338027e-2)**2,2)) + "e14 W/cm^2", fontsize=16)
    plt.plot(w_n[0:w_n_half], y_train_norm[i2], color='green')
    plt.xlabel('$\omega/\omega_0$', fontsize=14)
    plt.xlim((0, 70))
    plt.ylim((0, 1))    
    plt.ylabel('$\phi(\omega)$', fontsize=14)
    plt.xticks(np.arange(1, 3*Wmax/4, 2.0))   
    plt.grid()    
    plt.show() 
    plt.close()

In [ ]:
i_show1 = np.random.randint(0, n_train-1)
i_show2 = np.random.randint(0, n_train-1)
plot_train_norm_example(i_show1,i_show2)

## Model

In [ ]:
inputs = Input(shape=(3,))

x = Dense(16, activation='softplus')(inputs) 
x = Dense(32, activation='softplus')(x)                  
x = Reshape((32, 1))(x)
x = Conv1D(8, 4, activation='softplus', padding='same')(x)
x = Conv1D(8, 4, activation='softplus', padding='same')(x)
x = UpSampling1D(2)(x)
x = Conv1D(8, 4, activation='softplus', padding='same')(x)
x = Conv1D(8, 4, activation='softplus', padding='same')(x)
x = UpSampling1D(2)(x)
x = Conv1D(4, 4, activation='softplus', padding='same')(x) 
x = Conv1D(2, 4, activation='softplus', padding='same')(x)
x = Flatten()(x)
x = Dense(256, activation='softplus')(x)
outputs = Dense(w_n_half, activation='softplus')(x)

ModelGen = Model(inputs, outputs)
opt = Adam(lr=0.0005, amsgrad=True)
ModelGen.compile(optimizer=opt, loss='mean_squared_error') 

print(ModelGen.summary())

##Training

*   Training set: 30,000
*   Testing set: 1,000

In [ ]:
def plot_losses2():
    plt.figure(figsize=(8,4))
    plt.plot(np.log10(loss_sum),color='blue')
    plt.plot(np.log10(val_loss_sum),color='red')
    plt.title('Model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['training', 'validation'], loc='upper right')
    plt.show()

In [ ]:
tic = time.perf_counter()

for n in range(3, 10):       # training in a cycle with increasing batch size

      batch_size = 2**n

      history = ModelGen.fit(param_train_norm, y_train_norm, 
                             epochs=50,
                             batch_size=batch_size,
                             shuffle=True,
                             validation_data=(param_test_norm, y_test_norm))
      
      path = F"/model_thetaRI_FT/model_2" 
      ModelGen.save(path) 

      loss_save = history.history['loss']
      val_loss_save = history.history['val_loss']
      if n > 3:
            loss_sum = np.concatenate((loss_sum, loss_save), axis = 0)
            val_loss_sum = np.concatenate((val_loss_sum, val_loss_save), axis = 0)
      else:
            loss_sum = loss_save
            val_loss_sum = val_loss_save          
      
      plot_losses2()


toc = time.perf_counter()
print(f"Execution time {toc - tic:0.4f} seconds")
    

##Training and validation losses

In [ ]:
plot_losses2()  

##Prediction

In [ ]:
prediction_norm = ModelGen.predict(param_test_norm)
prediction = s_max*prediction_norm

##Function to draw the test and predicted examples

In [ ]:
def plot_examples(i1,i2):
    fig = plt.subplots(2,1,figsize=(12,9),constrained_layout=False)    
    plt.subplot(211)
    plt.title('Examples: Test points and prediction for parameters theta = ' + str(round(param_test[i1,0]*180/PI,2)) + ",  R = " + str(round(param_test[i1,1],2)) + \
             "a.u., I = " + str(round((param_test[i1,2]/5.338027e-2)**2,2)) + "e14 W/cm^2", fontsize=16)
    plt.plot(w_n, np.log10(y_test[i1]), color="blue", linewidth = 1)
    plt.plot(w_n[0:w_n_half], prediction[i1]+np.log10(s_min), color="red", linewidth = 1)
    plt.ylabel('$log(S(\omega))$', fontsize=16)
    plt.xticks(np.arange(0, 3*Wmax/4, 4.0))
    plt.xlim((0,100))
    plt.ylim((-16, -3))
    plt.grid()
    plt.subplot(212)
    plt.title('Examples: Test points and prediction for parameters theta = ' + str(round(param_test[i2,0]*180/PI,2)) + ",  R = " + str(round(param_test[i2,1],2)) + \
             "a.u., I = " + str(round((param_test[i2,2]/5.338027e-2)**2,2)) + "e14 W/cm^2", fontsize=16)
    plt.plot(w_n, np.log10(y_test[i2]), color="blue", linewidth = 1)
    plt.plot(w_n[0:w_n_half], prediction[i2]+np.log10(s_min), color="red", linewidth = 1) 
    plt.ylabel('$log(S(\omega))$', fontsize=16)
    plt.xlabel('$\omega/\omega_0$', fontsize=16) 
    plt.xticks(np.arange(0, 3*Wmax/4, 4.0))
    plt.xlim((0,100))
    plt.ylim((-16, -3))
    plt.grid()  
    plt.show() 
    plt.close()    

##Comparison of arbitrary $test$ (blue) and $decoded$ (red) curves

In [ ]:
i_show1 = np.random.randint(0, n_test-1)
i_show2 = np.random.randint(0, n_test-1)
plot_examples(i_show1, i_show2)